In [4]:
import pandas as pd
import logging
import random
import networkx as nx

In [57]:
class GraphConstructor:
    def __init__(
        self,
        followers_paths=["followers_Artificial_intelligence_Final.csv","followers_MachineLearning_Final.csv","followers_ChatGPT.csv"],
        data_paths=["data_Artificial_Intelligence.csv","data_chat_gpt.csv","data_MachineLearning.csv"],
        info_filepath="graph_info.json",
        centralities_filepath="centralities_info.json",
    ):
        
        self.df = pd.concat([
            pd.read_csv(followers_paths[0],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
            pd.read_csv(followers_paths[1],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
            pd.read_csv(followers_paths[2],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
        ])

       
        self.data = pd.concat([
            pd.read_csv(data_paths[0],dtype={"pk": str,"user_pk": str,"caption": str,"like_count": int},),
            pd.read_csv(data_paths[1],dtype={"pk": str,"user_pk": str,"caption": str,"like_count": int},),
            pd.read_csv(data_paths[2],dtype={"pk": str,"user_pk": str,"caption": str,"like_count": int},),
        ])
        
        self.graph = nx.DiGraph()
        self.info_filepath = info_filepath
        self.centralities_filepath = centralities_filepath

    def build_graph(self):
        # Raggruppa i follower per ciascun utente e aggiunge gli archi al grafo
        followers_per_user = (
            self.df.groupby("user_pk")["follower_pk"].apply(list).reset_index()
        )
        for _, (user_pk, listf) in followers_per_user.iterrows():
            for follower_pk in listf:
                self.graph.add_edge(follower_pk, user_pk)
        # Imposta una soglia casuale per ogni nodo
        for node in self.graph.nodes():
            self.graph.nodes[node]["threshold"] = random.uniform(0, 1)

        return self.graph

    def log_graph_info(self):
        logging.info(f"Numero di nodi del grafo: {self.graph.number_of_nodes()}")
        logging.info(f"Numero di archi del grafo: {self.graph.number_of_edges()}")

In [58]:
gc = GraphConstructor()
gc.build_graph()
graph = gc.graph
print(len(graph.nodes))
print(len(graph.edges))

347105
481447


In [63]:
def get_isolated_nodes(graph):
    return [node for node in graph.nodes if graph.degree(node) ==1]

isolated_nodes = get_isolated_nodes(graph)
print("Nodi isolati:", len(isolated_nodes))

Nodi isolati: 287804


In [74]:
followers_paths=["followers_Artificial_intelligence_Final.csv","followers_MachineLearning_Final.csv","followers_ChatGPT.csv"]
df = pd.concat([
            pd.read_csv(followers_paths[0],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
            pd.read_csv(followers_paths[1],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
            pd.read_csv(followers_paths[2],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
        ]).drop_duplicates()

df = df.groupby("user_pk")["follower_pk"].apply(list).reset_index()
df["follower_count"] = df["follower_pk"].str.len()
df = df.sort_values("follower_count", ascending=True)
df

,user_pk,follower_pk,follower_count
123,1368077759,"[22387437938, 51365695394]",2
577,28244257781,"[225195903, 55091225894, 67549867995]",3
72,11934491861,"[66409881116, 5443473681, 59139172299]",3
1056,48677968635,"[2378756552, 1427221100, 72037999721]",3
1672,71908864097,"[6864699219, 3136950333, 519835965, 529644941]",4
...,...,...,...
1370,60381420025,"[5788576235, 71134731317, 5460417708, 60811241...",6248
1561,67071502510,"[71134731317, 4699505254, 54105429395, 3096750...",6384
988,46560360663,"[228550097, 715582745, 502766522, 44176807210,...",7055
543,2671047146,"[5803179185, 65580853447, 63128560576, 4752437...",9473


In [27]:
num_rows = (df['follower_count'] < 100).sum()
print(num_rows)


1040


In [52]:
import pandas as pd

def add_new_edges_with_mapping(df: pd.DataFrame, new_edges: list) -> pd.DataFrame:
    # Precalcola una mappa per ogni follower_pk con i dati associati (prendiamo la prima occorrenza)
    mapping = (
        df[['follower_pk', 'follower_username', 'follower_count']]
        .drop_duplicates(subset='follower_pk')
        .set_index('follower_pk')
        .to_dict('index')
    )
    
    new_rows = []
    for i, (follower, user) in enumerate(new_edges, start=1):
        info = mapping.get(follower, {'follower_username': None, 'follower_count': 0})
        new_rows.append({
            'user_pk': user,
            'follower_pk': follower,
            'follower_username': info['follower_username'],
            'follower_count': info['follower_count']
        })
        # Puoi stampare il progresso se necessario
        if i % 1000 == 0:
            print(f"Aggiunto arco: {i}")
    
    new_df = pd.DataFrame(new_rows)
    # Combiniamo il DataFrame originale con quello delle nuove righe
    df_extended = pd.concat([df, new_df], ignore_index=True)
    return df_extended

# Esempio d'uso:
# 


def add_triadic_closure_edges(df: pd.DataFrame, max_new_edges: int = 100000) -> pd.DataFrame:
    follow_dict = {}
    for _, row in df.iterrows():
        follower = row['follower_pk']
        followed = row['user_pk']
        if follower not in follow_dict:
            follow_dict[follower] = set()
        follow_dict[follower].add(followed)
    
    existing_edges = set(zip(df['follower_pk'], df['user_pk']))
    new_edges = [] 
    reached_limit = False
    print("fine fase 1")
    i = 0
    for A, followed_set in follow_dict.items():
        if reached_limit:
            break
        for B in followed_set:
            if reached_limit:
                break
            if B in follow_dict:
                for C in follow_dict[B]:
                    if len(new_edges) >= max_new_edges:
                        reached_limit = True
                        break
                    if C not in followed_set and (A, C) not in existing_edges:
                        new_edges.append((A, C))
                        existing_edges.add((A, C)) 
                        i+=1
                        print("aggiunto arco", i)

    df_extended = add_new_edges_with_mapping(df_original, new_edges)
    return df_extended

# Esempio di utilizzo:
if __name__ == "__main__":
    followers_path = "followers_Artificial_Intelligence.csv" 
    df_original = pd.read_csv(
        followers_path,
        dtype={
            "user_pk": str,
            "follower_pk": str,
            "follower_username": str,
            "follower_count": int,
        }
    )
    
    print("Numero archi originali:", len(df_original))
    df_new = add_triadic_closure_edges(df_original, max_new_edges=75000)
    print("Numero archi dopo l'aggiunta:", len(df_new))


Numero archi originali: 312427
fine fase 1
aggiunto arco 1
aggiunto arco 2
aggiunto arco 3
aggiunto arco 4
aggiunto arco 5
aggiunto arco 6
aggiunto arco 7
aggiunto arco 8
aggiunto arco 9
aggiunto arco 10
aggiunto arco 11
aggiunto arco 12
aggiunto arco 13
aggiunto arco 14
aggiunto arco 15
aggiunto arco 16
aggiunto arco 17
aggiunto arco 18
aggiunto arco 19
aggiunto arco 20
aggiunto arco 21
aggiunto arco 22
aggiunto arco 23
aggiunto arco 24
aggiunto arco 25
aggiunto arco 26
aggiunto arco 27
aggiunto arco 28
aggiunto arco 29
aggiunto arco 30
aggiunto arco 31
aggiunto arco 32
aggiunto arco 33
aggiunto arco 34
aggiunto arco 35
aggiunto arco 36
aggiunto arco 37
aggiunto arco 38
aggiunto arco 39
aggiunto arco 40
aggiunto arco 41
aggiunto arco 42
aggiunto arco 43
aggiunto arco 44
aggiunto arco 45
aggiunto arco 46
aggiunto arco 47
aggiunto arco 48
aggiunto arco 49
aggiunto arco 50
aggiunto arco 51
aggiunto arco 52
aggiunto arco 53
aggiunto arco 54
aggiunto arco 55
aggiunto arco 56
aggiunto arco 

In [53]:
df_new.to_csv("followers_Artificial_Intelligence_Final.csv")

In [54]:
G = nx.from_pandas_edgelist(df_original, source='follower_pk', target='user_pk', create_using=nx.DiGraph())

# Conta e stampa il numero di nodi e archi
print("Numero di nodi:", G.number_of_nodes())
print("Numero di archi:", G.number_of_edges())

Numero di nodi: 278138
Numero di archi: 308745


In [55]:
G = nx.from_pandas_edgelist(df_new, source='follower_pk', target='user_pk', create_using=nx.DiGraph())

# Conta e stampa il numero di nodi e archi
print("Numero di nodi:", G.number_of_nodes())
print("Numero di archi:", G.number_of_edges())

Numero di nodi: 278138
Numero di archi: 383745
